'''
Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JoengYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.
'''

# Recyclable and Household Waste Classification 


https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification

In [1]:
import sys

path_append = "../" # Go up one directory from where you are.
sys.path.append(path_append) 

from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

In [2]:
dataset_path = '../data/Recyclable and Household Waste Classification/images/images'

In [3]:
import os
import torch
import torch.nn.functional as F
import random
from PIL import Image
from torch.utils.data import Dataset


class WasteDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []
        
        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for subfolder in ['default', 'real_world']:
                subfolder_dir = os.path.join(class_dir, subfolder)
                image_names = os.listdir(subfolder_dir)
                random.shuffle(image_names)
                
                if split == 'train':
                    image_names = image_names[:int(0.6 * len(image_names))]
                elif split == 'val':
                    image_names = image_names[int(0.6 * len(image_names)):int(0.8 * len(image_names))]
                else:  # split == 'test'
                    image_names = image_names[int(0.8 * len(image_names)):]
                
                for image_name in image_names:
                    self.image_paths.append(os.path.join(subfolder_dir, image_name))
                    self.labels.append(i)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(label, dtype=torch.long)
        label = F.one_hot(label, num_classes=30)
           
        return image, label

In [4]:
from torchvision import transforms

# Create the datasets and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = WasteDataset(path_append + dataset_path, split='train', transform=transform)
val_dataset = WasteDataset(path_append + dataset_path, split='val', transform=transform)
test_dataset = WasteDataset(path_append + dataset_path, split='test', transform=transform)

X, y = train_dataset[0]
print(X.shape)
print(y.shape)

torch.Size([3, 224, 224])
torch.Size([30])


In [5]:

data_config = DataConfig(dataset_name = 'recycle_image', task_type='multi_label_classification', obs_shape=[3, 224, 224], label_size=30)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'resnet', encoder_model = 'none')
ml_params.model.core_config.d_model = 256

In [6]:
from trainer_hub import TrainerHub

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)

In [7]:
trainer_hub.train(train_dataset, val_dataset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[0/100][50/140][Time 39.74]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.2223	Gen: 1.3845	Rec: 1.3507	E: 0.2561	R: 0.1885	P: 2.5129
--------------------Test Metrics------------------------
accuracy: 0.5741

[0/100][100/140][Time 38.63]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.1043	Gen: 1.2435	Rec: 1.2331	E: 0.1147	R: 0.0939	P: 2.3722
--------------------Test Metrics------------------------
accuracy: 0.4954



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[1/100][10/140][Time 38.67]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0989	Gen: 1.2228	Rec: 1.2053	E: 0.1164	R: 0.0815	P: 2.3291
--------------------Test Metrics------------------------
accuracy: 0.5780

[1/100][60/140][Time 41.51]
Unified LR across all optimizers: 0.00019815726328921765
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0878	Gen: 1.2059	Rec: 1.1929	E: 0.1009	R: 0.0747	P: 2.3110
--------------------Test Metrics------------------------
accuracy: 0.6521

[1/100][110/140][Time 43.13]
Unified LR across all optimizers: 0.00019770151423055492
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0701	Gen: 1.1878	Rec: 1.1790	E: 0.0790	R: 0.0613	P: 2.2967
--------------------Test Metrics------------------------
accuracy: 0.6195



...

Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[21/100][10/140][Time 39.54]
Unified LR across all optimizers: 0.0001745862334993144
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0598	Gen: 1.0356	Rec: 1.0367	E: 0.0587	R: 0.0609	P: 2.0126
--------------------Test Metrics------------------------
accuracy: 0.9540

[21/100][60/140][Time 39.49]
Unified LR across all optimizers: 0.00017418469630480507
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0579	Gen: 1.0337	Rec: 1.0355	E: 0.0560	R: 0.0597	P: 2.0113
--------------------Test Metrics------------------------
accuracy: 0.9359

[21/100][110/140][Time 39.50]
Unified LR across all optimizers: 0.00017378408262021616
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0599	Gen: 1.0344	Rec: 1.0353	E: 0.0590	R: 0.0608	P: 2.0097
--------------------Test Metrics------------------------
accuracy: 0.9520



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[22/100][20/140][Time 39.54]
Unified LR across all optimizers: 0.00017338439032153356
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0583	Gen: 1.0313	Rec: 1.0325	E: 0.0570	R: 0.0595	P: 2.0056
--------------------Test Metrics------------------------
accuracy: 0.9510

[22/100][70/140][Time 39.19]
Unified LR across all optimizers: 0.00017298561728962847
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0586	Gen: 1.0289	Rec: 1.0304	E: 0.0570	R: 0.0602	P: 2.0007
--------------------Test Metrics------------------------
accuracy: 0.9451

[22/100][120/140][Time 39.47]
Unified LR across all optimizers: 0.00017258776141024598
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0574	Gen: 1.0360	Rec: 1.0371	E: 0.0564	R: 0.0585	P: 2.0157
--------------------Test Metrics------------------------
accuracy: 0.9698



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]